# お惣菜（デリカ）AI エージェント ハンズオン

このハンズオンでは、Snowflake の AI 機能を使って**商品マスタ、POSデータ、在庫データ、気象データ、ウェブサーチ**の整備から **「自然言語で質問できる AI エージェント作成」** までを一気通貫で構築します。

### 今日つくるもの

お惣菜（唐揚げ・弁当・寿司・季節商品など）の **売上・在庫・気象データ** を横断的に分析できる AI エージェントです。  
完成すると、例えばこんな質問に自然言語で答えてくれます：

> 「2025年の唐揚げの売上推移を月別で可視化して。気温との相関も見たい」  
> 「現在欠品している商品と、その過去の売上トレンドを教えて」  
> 「うなぎ蒲焼きの売上が最も高い時期はいつ？」
> 「今週末に暑くなるらしいけど、何を多めに仕入れるべき？」

---

### ハンズオンの流れ（所要時間：約80分）

| Step | やること | 学べる Snowflake 機能 | 所要時間 |
|:----:|----------|----------------------|:--------:|
| 1 | AI で商品マスタを整備 | AI_CLASSIFY / AI_COMPLETE | 10分 |
| 2 | 商品の意味検索を可能にする | Cortex Search Service | 10分 |
| 3 | 気象データを取り込む | Snowflake Marketplace | 10分 |
| 4 | データの意味を定義する | Semantic View（Cortex Analyst） | 20分 |
| 5 | AI エージェントを組み立てる | Cortex Agent | 15分 |
| - | 動かしてみよう！ | - | 5分 |

## Step 0: 事前準備
使用するデータベース・スキーマ・ウェアハウスを設定します。

In [ ]:
USE DATABASE FOODEX_HANDSON;
USE SCHEMA ANALYTICS;
USE WAREHOUSE COMPUTE_WH;

---
## Step 1: AI で商品マスタを整備する

現在の商品マスタ（`RAW.PRODUCT_MASTER`）には、JANコード・商品名・メーカー名しかありません。  
ここに Snowflake の **AI SQL 関数** を使って、2つの情報を自動で追加します。

### 追加する情報

#### 1. 大カテゴリの自動分類 — `AI_CLASSIFY`

商品名だけを見て、AI が自動的にカテゴリを判定します。  
あらかじめ定義した **6つのカテゴリ** に分類します（MECE＝漏れなくダブりなく）。

| カテゴリ | どんな商品？ |
|----------|-------------|
| 揚げ物 | 唐揚げ、とんかつ、コロッケ、フライ類 |
| 和惣菜 | 煮物、きんぴら、ひじき、だし巻き卵 |
| 中華惣菜 | 麻婆豆腐、エビチリ、餃子、春巻き |
| 洋食惣菜 | パスタ、グラタン、ハンバーグ、カレー |
| 弁当・おにぎり・寿司 | 各種弁当、おにぎり、寿司類 |
| 季節商品 | 恵方巻き、おはぎ、うなぎ、おせちなど季節限定品 |

#### 2. 商品キャプションの自動生成 — `AI_COMPLETE`

商品名とメーカー名から、その商品の **魅力や特徴を伝える説明文（約300文字）** を AI が自動で生成します。

> **ポイント**: `AI_CLASSIFY` はラベル＋説明文を渡すだけで分類してくれるので、従来のルールベースのロジックを書く必要がありません。

In [ ]:
SELECT * FROM FOODEX_HANDSON.RAW.PRODUCT_MASTER;

それでは実行しましょう。商品数にもよりますが、**約1〜2分** で完了します。

In [ ]:
CREATE OR REPLACE TABLE FOODEX_HANDSON.ANALYTICS.AI_PRODUCT_MASTER AS
SELECT 
    JANCODE,
    PRODUCT_NAME,
    MAKER_NAME,
    MAKER_CODE,
    -- ① AI_CLASSIFY関数で大カテゴリを分類（MECE）
    AI_CLASSIFY(
        PRODUCT_NAME,
        [
            {'label': '揚げ物', 'description': '唐揚げ、とんかつ、コロッケ、フライ、天ぷら、メンチカツ、ハムカツ'},
            {'label': '和惣菜', 'description': '煮物、ひじき、きんぴら、切り干し大根、筑前煮、肉豆腐、卯の花、だし巻き卵、サラダ'},
            {'label': '中華惣菜', 'description': '麻婆豆腐、エビチリ、酢豚、八宝菜、回鍋肉、青椒肉絲、餃子、春巻き'},
            {'label': '洋食惣菜', 'description': 'パスタ、スパゲッティ、グラタン、ドリア、ハンバーグ、オムライス、カレー'},
            {'label': '弁当・おにぎり・寿司', 'description': '弁当、おにぎり、寿司、いなり寿司、ちらし寿司、のり弁、から揚げ弁当'},
            {'label': '季節商品', 'description': '恵方巻き、桜餅、柏餅、冷やし中華、うなぎ、おはぎ、月見団子、おでん、年越しそば、おせち、七草粥'}
        ],
        {'task_description': 'お惣菜・デリカ商品の商品名から最も適切な大カテゴリに分類してください'}
    ):labels[0]::VARCHAR AS CATEGORY,
    -- ② AI_COMPLETE関数で商品キャプション生成
    TRIM(SNOWFLAKE.CORTEX.AI_COMPLETE(
        'llama3.1-405b',
        '以下のお惣菜商品の魅力や特徴を伝えるキャプション（300文字程度）を日本語で1つ作成してください。
         商品名: ' || PRODUCT_NAME || '
         メーカー: ' || COALESCE(MAKER_NAME, '不明') || '
         キャプション:'
    )) AS PRODUCT_CAPTION,
    CURRENT_TIMESTAMP() AS CREATED_AT
FROM FOODEX_HANDSON.RAW.PRODUCT_MASTER;

### 結果を見てみましょう

AI が生成した `CATEGORY`（カテゴリ）と `PRODUCT_CAPTION`（商品キャプション）を確認します。

In [ ]:
SELECT * FROM FOODEX_HANDSON.ANALYTICS.AI_PRODUCT_MASTER;

### カテゴリ分類の精度を確認

6つのカテゴリにバランスよく分類されているか確認しましょう。  
偏りがある場合は、`AI_CLASSIFY` に渡すカテゴリの `description` を調整することで精度を改善できます。

In [ ]:
SELECT 
    CATEGORY,
    COUNT(*) AS PRODUCT_COUNT
FROM FOODEX_HANDSON.ANALYTICS.AI_PRODUCT_MASTER
GROUP BY CATEGORY
ORDER BY PRODUCT_COUNT DESC;

## Step 2: 気象データを取り込む（Snowflake Marketplace）

お惣菜の売上は天候に大きく左右されます。  
暑い日は冷やし中華やサラダが売れ、寒い日はおでんや煮物が売れる — こうした **天候×売上の分析** を可能にするため、  
Snowflake Marketplace の **気象データ** を取り込みます。

### データソース

**Weather Source LLC: Frostbyte** が提供するグローバル気象データを使用します。  
日本（JP）の日次気象データを 2019年〜 取得できます。

### なぜテーブルを作るのか？

元データは華氏・インチ単位で、グローバルデータが含まれています。  
POSデータと結合するには **日本のデータのみを抽出し、メートル法に変換して日付ごとに1行** にする必要があるため、  
全国平均に集約するテーブルを作成します。

| 集約カラム | 計算方法 | 説明 |
|------------|----------|------|
| 平均気温 | AVG | 全国の観測所の平均（℃に変換） |
| 最高気温 | MAX | 全国で最も高かった気温（℃に変換） |
| 最低気温 | MIN | 全国で最も低かった気温（℃に変換） |
| 降水量 | AVG | 全国平均の降水量（mmに変換） |
| 湿度 | AVG | 全国平均の湿度 |
| 日照時間 | AVG | 全国平均の日射量 |
| 降雪量 | SUM | 全国の降雪量合計（cmに変換） |
| 風速 | AVG | 全国平均の風速（km/hに変換） |

In [ ]:
CREATE OR REPLACE TABLE FOODEX_HANDSON.ANALYTICS.DAILY_WEATHER AS
SELECT
    DATE_VALID_STD                                                      AS WEATHER_DATE,
    ROUND(AVG((AVG_TEMPERATURE_AIR_2M_F - 32) * 5/9), 1)                AS AVG_TEMPERATURE,
    ROUND(MAX((MAX_TEMPERATURE_AIR_2M_F - 32) * 5/9), 1)                AS MAX_TEMPERATURE,
    ROUND(MIN((MIN_TEMPERATURE_AIR_2M_F - 32) * 5/9), 1)                AS MIN_TEMPERATURE,
    ROUND(AVG(TOT_PRECIPITATION_IN * 25.4), 1)                          AS AVG_RAINFALL,
    ROUND(AVG(AVG_HUMIDITY_RELATIVE_2M_PCT), 1)                         AS AVG_HUMIDITY,
    ROUND(AVG(AVG_RADIATION_SOLAR_TOTAL_WPM2), 2)                       AS AVG_SUNLIGHT_HOURS,
    ROUND(SUM(TOT_SNOWFALL_IN * 25.4), 1)                               AS TOTAL_SNOWFALL,
    ROUND(AVG(AVG_WIND_SPEED_10M_MPH * 1.60934), 1)                     AS AVG_WIND_SPEED
FROM WEATHER_SOURCE_LLC_FROSTBYTE.ONPOINT_ID.HISTORY_DAY
WHERE COUNTRY = 'JP'
GROUP BY DATE_VALID_STD
ORDER BY DATE_VALID_STD;

In [ ]:
SELECT * FROM FOODEX_HANDSON.ANALYTICS.DAILY_WEATHER;

---
## Step 3: 商品の意味検索を可能にする（Cortex Search）

Step 1 で作った `AI_PRODUCT_MASTER` に対して、**Cortex Search Service** を作成します。

### Cortex Search とは？

テキストの **意味** を理解して検索するサービスです。  
キーワードの完全一致ではなく、「こういう意味のものを探して」という検索ができます。

例えば「夏に食べたい冷たい麺」と聞くと、「冷やし中華」や「素麺」がヒットします。

### 仕組み

商品名・メーカー名・カテゴリ・AIキャプションを **1つのテキストに結合** し、  
Snowflake の Embedding モデル（`snowflake-arctic-embed-l-v2.0`）でベクトル化して検索可能にします。

In [ ]:
CREATE OR REPLACE CORTEX SEARCH SERVICE FOODEX_HANDSON.ANALYTICS.PRODUCT_SEARCH_SERVICE
    ON SEARCH_TEXT
    ATTRIBUTES JANCODE, PRODUCT_NAME, MAKER_NAME, CATEGORY, PRODUCT_CAPTION
    WAREHOUSE = COMPUTE_WH
    TARGET_LAG = '1 day'
    EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0'
AS (
    SELECT
        JANCODE,
        PRODUCT_NAME,
        MAKER_NAME,
        CATEGORY,
        PRODUCT_CAPTION,
        '商品名: ' || PRODUCT_NAME || 
        ' | メーカー: ' || COALESCE(MAKER_NAME, '') ||
        ' | カテゴリ: ' || COALESCE(CATEGORY, '') ||
        ' | 説明: ' || COALESCE(PRODUCT_CAPTION, '')
        AS SEARCH_TEXT
    FROM FOODEX_HANDSON.ANALYTICS.AI_PRODUCT_MASTER
);

---
## Step 4: データの意味を定義する（Semantic View）

**Semantic View（セマンティックビュー）** は、テーブルやカラムに **ビジネス上の意味** を付与し、  
AI が自然言語の質問を理解してSQLを生成できるようにする仕組みです。

### なぜ必要？

AI は `SALES_QTY` というカラム名だけでは「販売数量」という意味を理解できません。  
Semantic View で「SALES_QTY = 販売数量」と定義することで、  
「先月の売上数量は？」という質問に正しく答えられるようになります。

### 今回作成する Semantic View

| View名 | 説明 |
|--------|------|
| SV_SALES_ANALYSIS | 売上分析用（POS + 商品マスタ + 気象データを結合） |
| SV_INVENTORY | 在庫分析用 |

In [ ]:
CREATE OR REPLACE SEMANTIC VIEW FOODEX_HANDSON.ANALYTICS.SV_SALES_ANALYSIS

  TABLES (
    p AS FOODEX_HANDSON.ANALYTICS.POS_DATA
      PRIMARY KEY (SALE_DATE, JANCODE),
    m AS FOODEX_HANDSON.ANALYTICS.AI_PRODUCT_MASTER
      PRIMARY KEY (JANCODE),
    w AS FOODEX_HANDSON.ANALYTICS.DAILY_WEATHER
      PRIMARY KEY (WEATHER_DATE)
  )

  RELATIONSHIPS (
    p_to_m AS p (JANCODE) REFERENCES m,
    p_to_w AS p (SALE_DATE) REFERENCES w (WEATHER_DATE)
  )

  FACTS (
    p.SALES_QTY AS SALES_QTY COMMENT = 'quantity sold',
    p.SALES_AMOUNT AS SALES_AMOUNT COMMENT = 'total sales amount in Japanese Yen'
  )

  DIMENSIONS (
    p.SALE_DATE AS p.SALE_DATE COMMENT = 'the date when the sale occurred',
    p.SALE_YEAR AS YEAR(p.SALE_DATE) COMMENT = 'year of the sale',
    p.SALE_MONTH AS MONTH(p.SALE_DATE) COMMENT = 'month of the sale',
    p.JANCODE AS p.JANCODE COMMENT = 'JAN code',
    m.PRODUCT_NAME AS PRODUCT_NAME COMMENT = 'name of the product',
    m.MAKER_NAME AS MAKER_NAME COMMENT = 'manufacturer name',
    m.CATEGORY AS CATEGORY COMMENT = 'product category',
    w.AVG_TEMPERATURE AS AVG_TEMPERATURE COMMENT = 'average temperature in Celsius',
    w.MAX_TEMPERATURE AS MAX_TEMPERATURE COMMENT = 'maximum temperature in Celsius',
    w.AVG_RAINFALL AS AVG_RAINFALL COMMENT = 'average rainfall in mm'
  )

  METRICS (
    p.TOTAL_SALES AS SUM(p.SALES_AMOUNT) COMMENT = 'total sales amount',
    p.TOTAL_QTY AS SUM(p.SALES_QTY) COMMENT = 'total quantity sold'
  )

  COMMENT = 'お惣菜の売上分析用セマンティックビュー';

In [ ]:
CREATE OR REPLACE SEMANTIC VIEW FOODEX_HANDSON.ANALYTICS.SV_INVENTORY

  TABLES (
    i AS FOODEX_HANDSON.ANALYTICS.INVENTORY
      PRIMARY KEY (SNAPSHOT_DATE, JANCODE),
    m AS FOODEX_HANDSON.ANALYTICS.AI_PRODUCT_MASTER
      PRIMARY KEY (JANCODE)
  )

  RELATIONSHIPS (
    i_to_m AS i (JANCODE) REFERENCES m
  )

  FACTS (
    i.STOCK_QTY AS STOCK_QTY COMMENT = 'current stock quantity',
    i.STOCK_AMOUNT AS STOCK_AMOUNT COMMENT = 'total stock value in Japanese Yen'
  )

  DIMENSIONS (
    i.SNAPSHOT_DATE AS i.SNAPSHOT_DATE COMMENT = 'date when inventory was recorded',
    i.JANCODE AS i.JANCODE COMMENT = 'JAN code',
    i.STOCK_STATUS AS STOCK_STATUS COMMENT = 'stock status (通常, 残少, 欠品, 季節外)',
    m.PRODUCT_NAME AS PRODUCT_NAME COMMENT = 'name of the product',
    m.MAKER_NAME AS MAKER_NAME COMMENT = 'manufacturer name',
    m.CATEGORY AS CATEGORY COMMENT = 'product category'
  )

  METRICS (
    i.TOTAL_STOCK AS SUM(i.STOCK_QTY) COMMENT = 'total stock quantity',
    i.TOTAL_STOCK_VALUE AS SUM(i.STOCK_AMOUNT) COMMENT = 'total stock value'
  )

  COMMENT = 'お惣菜の在庫分析用セマンティックビュー';

### Semantic View のテスト

Cortex Analyst を使って、自然言語で質問してみましょう。

In [ ]:
-- 売上分析のテスト：カテゴリ別の売上金額
SELECT 
    カテゴリ,
    SUM(売上金額) AS 売上金額合計
FROM FOODEX_HANDSON.ANALYTICS.SV_SALES_ANALYSIS
WHERE 売上年 = 2025
GROUP BY カテゴリ
ORDER BY 売上金額合計 DESC;

In [ ]:
-- 在庫分析のテスト：欠品・残少の商品
SELECT 
    商品名,
    メーカー名,
    カテゴリ,
    在庫数量,
    在庫ステータス
FROM FOODEX_HANDSON.ANALYTICS.SV_INVENTORY
WHERE 在庫ステータス IN ('欠品', '残少')
ORDER BY 在庫ステータス, 在庫数量;

---
## Step 5: AI エージェントを組み立てる（Cortex Agent）

**Cortex Agent** は、複数のツール（Semantic View、Cortex Search、Web検索など）を  
**自律的に組み合わせて** 質問に答える AI エージェントです。

### エージェントが使えるツール

| ツール | 用途 |
|--------|------|
| SV_SALES_ANALYSIS | 売上データの分析（SQL生成） |
| SV_INVENTORY | 在庫データの分析（SQL生成） |
| PRODUCT_SEARCH_SERVICE | 商品の意味検索 |
| Web検索 | 最新情報の取得（新商品情報など） |

### エージェントの動作イメージ

質問: 「2025年の唐揚げの売上を月別で見せて」

1. エージェントが質問を理解
2. `SV_SALES_ANALYSIS` を選択
3. SQLを自動生成・実行
4. 結果を自然言語で回答

In [ ]:
CREATE OR REPLACE AGENT FOODEX_HANDSON.ANALYTICS.OSOZAI_AGENT
  COMMENT = 'お惣菜の売上・在庫・商品情報を分析するAIエージェント'
  FROM SPECIFICATION
  $$
  models:
    orchestration: claude-3-5-sonnet

  instructions:
    system: |
      あなたはお惣菜（デリカ）専門の分析アシスタントです。
      以下のデータにアクセスできます：

      1. 売上データ（2022年〜2026年）：POSデータと気象データが結合済み
      2. 在庫データ：現在の在庫状況
      3. 商品マスタ：94商品（唐揚げ、弁当、寿司、季節商品など）

      質問に対して、適切なツールを選択し、正確なデータに基づいて回答してください。
      金額は日本円で表示し、必要に応じてグラフや表を提案してください。

      季節商品について：
      - 恵方巻き（2月節分）、桜餅（3-4月）、柏餅（5月）
      - 冷やし中華・素麺（6-8月）、うなぎ（土用の丑）
      - おはぎ（お彼岸）、月見団子（9月）、さんま・栗ご飯・松茸ご飯（秋）
      - おでん（10-3月）、年越しそば・おせち・お雑煮（年末年始）、七草粥（1月7日）

  tools:
    - tool_spec:
        type: cortex_analyst_text_to_sql
        name: SalesAnalysis
        description: 売上データの分析（POS、商品マスタ、気象データ結合）
    - tool_spec:
        type: cortex_analyst_text_to_sql
        name: InventoryAnalysis
        description: 在庫データの分析
    - tool_spec:
        type: cortex_search
        name: ProductSearch
        description: 商品の意味検索

  tool_resources:
    SalesAnalysis:
      semantic_view: FOODEX_HANDSON.ANALYTICS.SV_SALES_ANALYSIS
    InventoryAnalysis:
      semantic_view: FOODEX_HANDSON.ANALYTICS.SV_INVENTORY
    ProductSearch:
      name: FOODEX_HANDSON.ANALYTICS.PRODUCT_SEARCH_SERVICE
      max_results: 10
  $$

---
## 🎉 完成！動かしてみよう

Snowsight の **Cortex Agent** 画面から、作成したエージェントを試してみましょう。

### サンプル質問

#### 売上分析
- 「2025年のカテゴリ別売上ランキングを教えて」
- 「唐揚げの月別売上推移を可視化して」
- 「気温が30度以上の日に売れる商品は？」
- 「うなぎ蒲焼きの売上が最も高い時期はいつ？」

#### 在庫分析
- 「現在欠品している商品を教えて」
- 「在庫金額が高い商品トップ10は？」
- 「残少の商品とその売上トレンドを教えて」

#### 商品検索
- 「暑い日におすすめの商品を探して」
- 「お花見にぴったりな商品は？」
- 「ヘルシーな和惣菜を教えて」

#### 複合質問
- 「来週末に気温が上がるらしいけど、何を多めに仕入れるべき？」
- 「節分の恵方巻きの売上推移と、来年の予測を教えて」

In [ ]:
-- エージェントをSQLから呼び出す例
SELECT SNOWFLAKE.CORTEX.AGENT(
    'FOODEX_HANDSON.ANALYTICS.DELI_AGENT',
    '2025年の揚げ物カテゴリの月別売上を教えてください'
);

---
## おまけ：季節性の分析

季節商品の売上パターンを確認してみましょう。

In [ ]:
-- 季節商品の月別売上
SELECT 
    商品名,
    売上月,
    SUM(販売数量) AS 販売数量合計,
    SUM(売上金額) AS 売上金額合計
FROM FOODEX_HANDSON.ANALYTICS.SV_SALES_ANALYSIS
WHERE カテゴリ = '季節商品'
  AND 売上年 = 2025
GROUP BY 商品名, 売上月
HAVING SUM(販売数量) > 0
ORDER BY 商品名, 売上月;

In [ ]:
-- 気温と売上の相関（サラダ類）
SELECT 
    CASE 
        WHEN 平均気温 < 10 THEN '10℃未満'
        WHEN 平均気温 < 20 THEN '10-20℃'
        WHEN 平均気温 < 25 THEN '20-25℃'
        WHEN 平均気温 < 30 THEN '25-30℃'
        ELSE '30℃以上'
    END AS 気温帯,
    AVG(販売数量) AS 平均販売数量
FROM FOODEX_HANDSON.ANALYTICS.SV_SALES_ANALYSIS
WHERE 商品名 LIKE '%サラダ%'
GROUP BY 気温帯
ORDER BY 気温帯;